In [1]:
!pip install gdown


In [ ]:
import cv2
import numpy as np
from scipy.fft import fft
import matplotlib.pyplot as plt
import gdown
import os


def calculate_frequency(signal, fps):
    n = len(signal)
    freq = np.fft.fftfreq(n, d=1/fps)
    fft_values = fft(signal)
    positive_freq = freq[freq > 0]
    magnitude = np.abs(fft_values[0:len(positive_freq)])
    dominant_freq_index = np.argmax(magnitude)
    dominant_freq = positive_freq[dominant_freq_index]
    return dominant_freq

# Function to find connected components with a given frequency range
def find_components_with_frequency(video_path, frequency_range=(0, 1), connectivity=4):
   
    output_path = "thermal_video.mp4"
    gdown.download(video_path, output_path, quiet=False)

    cap = cv2.VideoCapture(output_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    component_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frequency = calculate_frequency(gray.flatten(), fps)

        if frequency_range[0] <= frequency <= frequency_range[1]:
            _, labels = cv2.connectedComponents(gray, connectivity=connectivity)
            component_list.append(labels)

            frame_with_components = cv2.applyColorMap((labels * 15).astype(np.uint8), cv2.COLORMAP_VIRIDIS)
            cv2.imshow("Connected Components", frame_with_components)
            if cv2.waitKey(30) & 0xFF == 27: 
                break

    cap.release()
    cv2.destroyAllWindows()
    os.remove(output_path)  
    return component_list

video_drive_link = "https://drive.google.com/uc?id=1PWS2MoFphHwTwblN82QZZbYqEmycN0Jj"
components = find_components_with_frequency(video_drive_link, frequency_range=(0, 1), connectivity=4)
